# Gesundheitsausgaben-Analyse

## Projektübersicht
Dieses Projekt analysiert Gesundheitsausgaben basierend auf verschiedenen Kategorien wie Finanzierungsquellen, Arten von Gesundheitsleistungen und Zeiträumen.

Ziel ist es, Einblicke in die Entwicklung der Gesundheitskosten zu gewinnen und Muster zu identifizieren, die zukünftige Entscheidungen in der Gesundheitspolitik beeinflussen können.

### **Projektziele:**
1. **Datenbereinigung:** Aufbereitung der Rohdaten durch Entfernen unnötiger Spalten.
2. **Datenanalyse:** Identifikation von Trends in den Gesundheitsausgaben.
3. **Visualisierung:** Erstellung von Diagrammen für die Interpretation.
4. **Berichterstellung:** Zusammenfassung der wichtigsten Erkenntnisse.

### **Verwendete Datensätze:**
Die Analyse basiert auf folgenden CSV-Dateien:
- `OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0.csv` (Gesundheitsleistungen und -güter)
- `OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0.csv` (Zeitreihen der Gesundheitsausgaben)
- `OGD_gesausgaben01_HVD_HCHF_1_HEADER.csv` (Metadaten über Spaltenbezeichnungen)

### **Projektstruktur:**
- `data/` → Enthält die Anfangsdaten
- `notebooks/` → Enthält das Jupyter Notebook
- `output/` → Speichert bereinigte und aggregierte Daten
- `scripts/` → Enthält wiederverwendbare Python-Funktionen

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## 1. Datenanalyse

1. **Hauptdatensatz:**  
   - Datei: `OGD_gesausgaben01_HVD_HCHF_1.csv`  
   - Enthält die eigentlichen Gesundheitsausgaben-Daten.  

2. **Hilfsdatensätze:**  
   - `OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0.csv` – Gesundheitsleistungen und -güter (HC)  
   - `OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0.csv` – Zeitreihen der Gesundheitsausgaben  
   - `OGD_gesausgaben01_HVD_HCHF_1_HEADER.csv` – Header-Metadaten zur Beschreibung der Spalten  

#### Warum verwenden wir `;` als Delimiter?  

Die bereitgestellten CSV-Dateien verwenden das Semikolon (`;`) als Trennzeichen (wir glauben um Verwechslungen mit Dezimalpunkten oder -kommas zu vermeiden)

Daher muss beim Einlesen `delimiter=';'` explizit angegeben werden, um die Daten korrekt zu parsen.  


In [ ]:
def read_dfs(file_path="data"):
    dataframes = {}
    for filename in os.listdir(file_path):
        if not filename.endswith('.csv'):
            continue
        file_path = os.path.join('data', filename)
        df_ = pd.read_csv(file_path, delimiter=';')
        df_name = filename.replace('.csv', '')
        dataframes[df_name] = df_
    return dataframes

dataframes = read_dfs()
df_main = dataframes['OGD_gesausgaben01_HVD_HCHF_1']
df_hc = dataframes['OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0']
df_time = dataframes['OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0']
df_header = dataframes['OGD_gesausgaben01_HVD_HCHF_1_HEADER']

def analyze_dataframe(df, name):
    print(f"\n--- {name} ---")
    
    print("\nErste 5 Zeilen:")
    display(df.head())

    # zufällige Stichprobe
    print("\nZufällige Stichprobe:")
    display(df.sample(5))

    # Überblick über die Struktur des DataFrames
    print("\nDateninfo:")
    df.info()

    # Deskriptive Statistiken
    print("\nDeskriptive Statistiken (numerische und kategoriale Werte):")
    display(df.describe())

    # Unique Werte für jede Spalte (nur für kategorische Spalten sinnvoll)
    print("\nAnzahl eindeutiger Werte pro Spalte:")
    for col in df.columns:
        unique_values = df[col].nunique()
        print(f"{col}: {unique_values} eindeutige Werte")
        # print(f"  Werte: {df[col].unique()}")  # unnötig zu viel output
        
    # ganz wichtig noch null werte!
    print(f"Fehlende Werte in {name}:")
    display(df.isnull().sum())

for df_name, df in dataframes.items():
    analyze_dataframe(df, df_name)

### Ergebnisse:  

**1. Hauptdatensatz (Gesundheitsausgaben)**  
- **Anzahl:** 729 Zeilen, 16 Spalten.  
- **Datentypen:** Alle Spalten sind vom Typ `object`. Dies deutet darauf hin, dass möglicherweise numerische Werte als Text gespeichert wurden und eine Umwandlung notwendig sein könnte.  
- **Fehlende Werte:** Keine fehlenden Werte festgestellt.  
- **Mögliche Maßnahmen:** Überprüfung der Datenformate (z. B. Zahlen als Text) und mögliche Konvertierung in numerische Typen.  

---

**2. Gesundheitsleistungen und -güter (HC)**  
- **Anzahl:** 46 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Spalten `de_desc`, `de_link`, `en_desc`, `en_link`, `de_syn`, `en_syn` enthalten ausschließlich fehlende Werte und sollten entfernt werden.  
  - Die Spalte `FK` weist 7 fehlende Werte auf, welche genauer überprüft werden müssen (z. B. Kategorie oder Obergruppe).  
- **Mögliche Maßnahmen:**  
  - Löschen irrelevanter Spalten mit ausschließlich fehlenden Werten.  
  - Umgang mit fehlenden Werten in `FK` (Ersatz, Gruppierung oder Löschung).  

---

**3. Zeitreihen der Gesundheitsausgaben**  
- **Anzahl:** 19 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Die Spalte `Unnamed: 2` enthält nur `NaN`-Werte und sollte entfernt werden.  
  - Weitere Spalten (`de_desc`, `de_link`, etc.) sind ebenfalls vollständig leer.  
- **Datentypen:**  
  - Die Spalten `name` und `en_name` sind als `int64` klassifiziert – sollte überprüft werden, ob dies korrekt ist oder in `string`/`category` geändert werden muss.  
- **Mögliche Maßnahmen:**  
  - Entfernen leerer Spalten.  
  - Datentypen anpassen, falls erforderlich.  

---

**4. Header-Metadaten**  
- **Anzahl:** 16 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Unbenannte Spalten `Unnamed: 3` bis `Unnamed: 9` enthalten ausschließlich fehlende Werte und können entfernt werden.  
- **Mögliche Maßnahmen:**  
  - Entfernen leerer Spalten zur Reduzierung unnötiger Daten.

## 2. Datenbereinigung

**Beobachtungen zur Anzahl fehlender Werte:**  

- Der Hauptdatensatz enthält **keine** fehlenden Werte.  
- Der HC-Datensatz weist mehrere Spalten mit vollständig fehlenden Werten auf.  
- Unbenannte Spalten in den Zeitreihen- und Header-Daten enthalten nur NaN-Werte.  

**TODO:**  
- Entfernen unnötiger Spalten mit vollständig fehlenden Werten.  

In [ ]:
df_hc_cleaned = df_hc.drop(columns=['de_desc', 'de_link', 'en_desc', 'en_link', 'de_syn', 'en_syn'])
df_time_cleaned = df_time.drop(columns=['Unnamed: 2', 'de_desc', 'de_link', 'en_desc', 'en_link', 'de_syn', 'en_syn'])
df_header_cleaned = df_header.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'])

print("Bereinigter HC-Datensatz:")
display(df_hc_cleaned.sample(5))

print("Bereinigter Zeitreihen-Datensatz:")
display(df_time_cleaned.head())

print("Bereinigter Header-Datensatz:")
display(df_header_cleaned.head())

### FK-Spalte (Fremdschlüssel)

Die FK-Spalte ist in einigen Zeilen leer (`NaN`), was darauf hindeutet, dass diese Zeilen übergeordnete Kategorien sind.  
Unser Ziel ist es, die Hierarchie der Gesundheitsleistungen zu verstehen und ggf. geeignete Anpassungen vorzunehmen.

**Zu untersuchende Aspekte:**

1. Anzahl der `NaN`- und nicht-`NaN`-Werte in der FK-Spalte.  
2. Identifikation der übergeordneten Kategorien anhand der FK-Spalte.  

In [ ]:
# Anzahl der fehlenden Werte in der FK-Spalte
missing_fk = df_hc['FK'].isnull().sum()
total_rows = len(df_hc)

print(f"Anzahl der fehlenden FK-Werte (Überkategorien): {missing_fk}")
print(f"Anzahl der vorhandenen FK-Werte (Unterkategorien): {total_rows - missing_fk}")

# Anzeigen der Zeilen mit fehlender FK (Überkategorien)
df_hc_missing_fk = df_hc[df_hc['FK'].isnull()]
display(df_hc_missing_fk[['code', 'name']])

# Anzeigen der Zeilen mit vorhandener FK (Unterkategorien)
df_hc_with_fk = df_hc[df_hc['FK'].notnull()]
display(df_hc_with_fk[['code', 'name', 'FK']].head(10))

**Ergebnisse der FK-Analyse:**  

- Einträge mit fehlender FK markieren übergeordnete Kategorien.  
- Einträge mit vorhandener FK verweisen auf ihre jeweilige Oberkategorie.  
- Diese Hierarchie hilft uns, die Daten später besser zu analysieren und zu gruppieren.  

**Nächster Schritt:**  
- Erstellung einer neuen Spalte `Kategorieebene`, die zwischen übergeordneten und untergeordneten Kategorien unterscheidet.  
- Falls sinnvoll, Speicherung einer hierarchischen Struktur für spätere Analysen oder Visualisierungen.  


In [ ]:
# Neue Spalte zur Kategorisierung hinzufügen (Über-/Unterkategorie)
df_hc['Kategorieebene'] = df_hc['FK'].apply(lambda x: 'Oberkategorie' if pd.isnull(x) else 'Unterkategorie')

# anzeigen der kartegorieebene
display(df_hc[['code', 'name', 'FK', 'Kategorieebene']].head(15))

# anzahl der Ober- und Unterkategorien anzeigen
print(df_hc['Kategorieebene'].value_counts())


### Bereinigung der Sprach- und Null-Spalten

In den Datensätzen sind Spalten in deutscher und englischer Sprache enthalten.  
Da die englischen Spalten redundant sind, werden sie entfernt.  
Zusätzlich werden alle vollständig leeren Spalten identifiziert und aus den Daten gelöscht.

#### Schritte:
1. Entfernen der englischen Spalten.
2. Identifikation von komplett leeren Spalten.
3. Löschen der leeren Spalten, um die Datenqualität zu verbessern.
4. Überprüfung der Datenstruktur nach der Bereinigung.


In [ ]:
def clean_df(dataframe, name):
    print(f"\n--- Bereinigung für {name} ---")

    # 1. Englische Spalten entfernen (beginnen mit 'en_')
    df_cleaned = df.drop(columns=[col for col in df.columns if col.startswith('en_')])

    # 2. Vollständig leerer Spalten
    empty_cols = df_cleaned.columns[df_cleaned.isnull().all()].tolist()
    print(f"Leere Spalten in {name}: {empty_cols}")

    # 3. Entfernen ovn leeren Spalten
    df_cleaned.drop(columns=empty_cols, inplace=True)

    # 4. Shape (Rows, Cols) nach Bereinigung
    print(f"Nach Bereinigung der englischen und leeren Spalten - {name}:")
    print(f"{name} shape:", df_cleaned.shape)

    # 5. Speichern der bereinigten DataFrames
    file_name = name + "_corr.csv"
    df_cleaned.to_csv(f'data/corr/{file_name}', index=False, sep=';')
    print(f"Bereinigte Datei für {name} gespeichert als {file_name}")

for name, df in dataframes.items():
    clean_df(df, name)

### Mergen der DataFrames und Datenvorbereitung

Um die verschiedenen DataFrames mit relevanten Informationen zu kombinieren, haben wir alle DataFrames anhand des gemeinsamen Schlüssels `code` zusammengeführt. Dies stellt sicher, dass alle relevanten Daten in einem einzigen DataFrame vorliegen und keine Informationen verloren gehen.

#### Schritte:
1. **Mergen der DataFrames**:
   - Wir haben die DataFrames `df_main`, `df_hc`, `df_time` und `df_header` mithilfe des gemeinsamen Schlüssels `code` zusammengeführt.
   - Ein `left join` wurde verwendet, um sicherzustellen, dass alle Zeilen aus dem Hauptdatensatz (`df_main`) erhalten bleiben, und nur passende Zeilen aus den anderen DataFrames hinzugefügt werden.

2. **Datenvorbereitung (Preprocessing)**:
   - **Extraktion der numerischen Werte**: Alle numerischen Spalten (z.B. `int64`, `float64`) wurden extrahiert und in einer separaten CSV-Datei gespeichert.
   - **Normalisierung der numerischen Werte**: Die numerischen Werte wurden mit einer Min-Max-Normalisierung skaliert und ebenfalls in einer separaten CSV-Datei gespeichert.
   - **Extraktion der nominalen (kategorischen) Werte**: Alle nominalen Werte (z.B. `object`, `category`) wurden extrahiert und ebenfalls in einer separaten CSV-Datei gespeichert.

#### Ergebnis:
- **df_merged_num.csv**: Enthält die numerischen Werte aus allen zusammengeführten DataFrames.
- **df_merged_norm.csv**: Enthält die normalisierten numerischen Werte.
- **df_merged_nom.csv**: Enthält die nominalen (kategorischen) Werte.

Das Mergen und die Datenvorbereitung sorgen dafür, dass alle relevanten Daten an einem Ort vorhanden sind und für die spätere Analyse bereitgestellt werden.


In [ ]:
# Daten einlesen
df_main = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_corr.csv', delimiter=';')
df_hc = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0_corr.csv', delimiter=';')
df_time = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0_corr.csv', delimiter=';')
df_header = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_HEADER_corr.csv', delimiter=';')

# 1. Mergen der DataFrames (angenommen, 'code' ist der gemeinsame Schlüssel)
merged_df = pd.merge(df_hc_corr, df_time_corr, on='code', how='left')  # 'inner' für den Schnitt der beiden DataFrames
final_merged_df = pd.merge(merged_df, df_header_corr, on='code', how='left')

# Überprüfen der ersten Zeilen des finalen DataFrames
display(final_merged_df)


# Funktion zur Bereinigung und Speicherung der Daten
def preprocess_and_save(df, filename_prefix):
    # 1. Numerische Werte extrahieren
    df_numeric = df.select_dtypes(include=['int64', 'float64'])

    # Speichern der numerischen Werte in 'num.csv'
    df_numeric.to_csv(f'{filename_prefix}_num.csv', index=False, sep=';')

    # 2. Normalisierung der numerischen Werte (Min-Max Normalisierung)
    df_normalized = (df_numeric - df_numeric.min()) / (df_numeric.max() - df_numeric.min())

    # Speichern der normalisierten Werte in 'norm.csv'
    df_normalized.to_csv(f'{filename_prefix}_norm.csv', index=False, sep=';')

    # 3. Nominale Werte extrahieren
    df_nominal = df.select_dtypes(include=['object', 'category'])

    # Speichern der nominalen Werte in 'nom.csv'
    df_nominal.to_csv(f'{filename_prefix}_nom.csv', index=False, sep=';')

# Anwendung auf das gemergte DataFrame
# preprocess_and_save(df_merged, 'df_merged')

print("Daten wurden erfolgreich gespeichert.")
